In [ ]:
episodes, scores, frames, start_episode, steps_done = [], [], [], 0, 0

In [ ]:
for i_episode in range(start_episode, start_episode+100):
    frame, _= env.reset()
    frame = resize_frame(frame)
    # We start with 30 no-op actions
    for i in range(no_op_action):
        starting_frame = resize_frame(env.step(0)[0])
    i=0
    while i<3:
        frames.append(starting_frame)
        i+=1
    state = get_state([frames[-3], frames[-2], frames[-1], frame])

    for t in range(1000):
        with torch.no_grad():
            action = agent_explore(state)
        frame, reward, terminated, truncated, _= env.step(action)
        #rescaled, and gray-scale, version of the last four frames
        frame = resize_frame(frame)
        frames.append(frame)
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = get_state([frames[-3], frames[-2], frames[-1], frame])

        memory.push(state, action, next_state, reward)

        state = next_state

        if steps_done >= INITIAL_MEMORY and steps_done % UPDATE_FREQ == 0:
            # The memory gets sampled to update the network every 4
            # steps with minibatches of size 32
            # Every 4 actions, a gradient descent step is performed and Q-values are updated
            optimize_model()

        if steps_done % TARGET_UPDATE == 0:
            update_target_network(policy_net, target_net, TAU)

        if done:
            break

    scheduler.step()
    done = False
    score, i = 0, 0
    frame, _= env.reset()
    frame = resize_frame(frame)
    starting_frame = resize_frame(env.step(0)[0])
    while i in (0,3):
        frames.append(starting_frame)
        i+=1
    state=get_state([frames[-3], frames[-2], frames[-1], frame])
    for t in range(1000):
        state = state.unsqueeze(0)
        action = policy_net(state).max(1)[1].view(1, 1)
        frame, reward, terminated, truncated, _= env.step(action.item())
        print("actions throughout steps and episodes", action.item(), t, i_episode)
        frame = resize_frame(frame)
        frames.append(frame)
        score += reward
        done = terminated or truncated

        if not done:
            next_state = get_state([frames[-3], frames[-2], frames[-1], frame])
        else:
            next_state=None
        state = next_state
        if done:
            break

    plot_durations()
    episodes.append(i_episode)
    scores.append(score)

    if (i_episode+1) % 5000 == 0:
        #We store only the minimum number of frames to train the next state
        frames = frames[-4:-1]
        print(f"Saving weights at Episode {i_episode+1} ...")
        torch.save(model.state_dict(), model_file_path)
        torch.save(data, data_file_path)

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()